## **Предпроцессинг (художественной) литературы и CBOW для ассирийского**


    объединить все файлы найденные в один
    сделать так, чтобы на каждой строчке по предложению
    к нижнему регистру
    убрать знаки препинания и лишнее всё, что будет
    убрать стоп слова/ короткие слова
    токенизировать
    лемматизировать (через лемматизатор)
    перепроверить, почистить остатки

    создать корпус и словарь
    cbow



In [ ]:
!pip install nltk
!pip install uniparser-urmi
!pip install gensim

#### **1. Объединяем файлы в один, где каждое предложение на отдельной строчке.**
- удаляем пунктуацию
- всё приводим к нижнему регистру

In [ ]:
import os
import re


def remove_punctuation(text):
    """
    Удаляет пунктуацию из текста, оставляя слова с апострофом.
    Если апостроф - второй символ в слове, заменяет его пробелом.
    """
    def replace_apostrophe(match):
        word = match.group(0)
        if len(word) > 1 and word[1] == '’':   # Обрабатывает '
            return word[0] + " " + word[2:]
        else:
            return word
    
    pattern = r"\b\w*’\w*\b"
    text = re.sub(pattern, replace_apostrophe, text)
    return re.sub(r'[^\w\s’]', ' ', text)

def combine_files_into_sentences_1(folder_path, output_file):
    """
    Объединяет текстовые файлы из папки в один файл,
    где каждое предложение находится на новой строке.
    Разделение на предложения происходит по знакам .?!
    """

    all_text = ""
    for filename in os.listdir(folder_path): # из списка всех файлов в папке*
        if filename.endswith(".txt"):
            filepath = os.path.join(folder_path, filename) # создаёт путь к файлу
            try:
                with open(filepath, 'r', encoding='utf-8') as file:
                    all_text += file.read()
            except UnicodeDecodeError:
                print(f"Ошибка кодировки при чтении файла: {filename}. Пожалуйста, убедитесь, что файл в кодировке UTF-8")
                continue
                
    # Разделение на предложения по знакам . ? !
    all_text = all_text.replace('\n', ' ') # для стихов*
    sentences = re.findall(r'[^.?!]+[.?!]?', all_text)
    #print(sentences)

    with open(output_file, 'w', encoding='utf-8') as outfile:
        for sentence in sentences:
            cleaned_sentence = remove_punctuation(sentence) # удаляем пунктуацию после того, как разбили на предложения
            lower_sentence = cleaned_sentence.lower()
            print(lower_sentence)
            outfile.write(lower_sentence.strip() + '\n')

if __name__ == "__main__":
  folder = '/home/kuklelik/Jupyter/ассирийские файлы/TXT_оригиналы/'
  output_file = 'output_all.txt'
  combine_files_into_sentences_1(folder, output_file)
  print(f"Файлы объединены в '{output_file}'.")

#### **2.1 Сохраняем полные разборы предложений (отдельные разобранные слова)**

In [ ]:
import os
import re
from uniparser_urmi import UrmiAnalyzer
import logging

# Настройка логгирования
logging.basicConfig(filename='error.log', level=logging.ERROR, format='%(asctime)s - %(levelname)s - %(message)s')


def tokenize_and_lemmatize(input_file, output_file):
    """
    Токенизирует и лемматизирует предложения из файла.
    Если лемма не найдена для слова, начинающегося на "в",
    удаляет "в" и пробует ещё раз.
    """

    a = UrmiAnalyzer(mode='nodiacritics')
    sentences = []
    with open(input_file, 'r', encoding='utf-8') as infile:
        for line in infile:
            line = line.strip()
            if line:
                sentences.append(line)

    results = []
    for sentence in sentences:
        try:
            lemmatized_sentence = []
            words = sentence.split()
            for word in words:
                analyses = a.analyze_words(word)
                lemmatized_words = []
                lemma_found = False  # Флаг для отслеживания найденной леммы

                for ana in analyses:
                    if ana.lemma:  # если лемма не пустая, добавляем анализ
                        lemmatized_words.append({
                            "token": ana.wf,
                            "lemma": ana.lemma,
                            "gramm": ana.gramm,
                        })
                        lemma_found = True
                        break

                if not lemma_found and word.lower().startswith('в'):  # Проверка в
                    modified_word = word[1:]
                    analyses = a.analyze_words(modified_word)
                    for ana in analyses:
                        if ana.lemma:
                            lemmatized_words.append({
                                "token": ana.wf,
                                "lemma": ana.lemma,
                                "gramm": ana.gramm,
                            })
                            lemma_found = True
                            break
                
                if not lemma_found: # если лемма так и не найдена, выводим пустую лемму
                    lemmatized_words.append({
                                "token": word,
                                "lemma": "", # пустая лемма
                                "gramm": "", # можно и грамматику сделать пустой
                            })


                lemmatized_sentence.append({
                    "word": word,
                    "analyses": lemmatized_words,
                })

            results.append({
                "sentence": sentence,
                "lemmatized_words": lemmatized_sentence,
            })
        except Exception as e:
            logging.error(f"Ошибка при обработке предложения '{sentence}': {e}")
            results.append({
                "sentence": sentence,
                "lemmatized_words": [],
            })

    with open(output_file, 'w', encoding='utf-8') as outfile:
       for item in results:
            outfile.write(f"Предложение: {item['sentence']}\n")
            for word_info in item['lemmatized_words']:
                outfile.write(f"  Слово: {word_info['word']}\n")
                for analysis in word_info["analyses"]:
                    outfile.write(f"    Токен: {analysis['token']}\n")
                    outfile.write(f"      Лемма: {analysis['lemma']}\n")
                    outfile.write(f"      Грамматика: {analysis['gramm']}\n")
            outfile.write("\n")

if __name__ == "__main__":
    input_file = 'output_all.txt'
    output_file = 'output_lemmatized_all.txt'
    tokenize_and_lemmatize(input_file, output_file)
    print(f"Токенизация и лемматизация завершены. Результаты сохранены в '{output_file}'.")

#### **2.2 Разбираемся с нелемматизированным остатком!**

In [ ]:
def find_words_without_lemma(input_file, output_file):
    """
    Файл с not_lemma
    Находит и записывает в файл слова без лемм (>= 4 символов) из файла output_lemmatized_all.txt
    """

    words_without_lemma = []
    current_word = None
    lemma_found = False

    with open(input_file, 'r', encoding='utf-8') as infile:
        for line in infile:
            line = line.strip()

            if line.startswith("Слово:"):
                if current_word and not lemma_found and len(current_word) >= 4:
                    words_without_lemma.append(current_word)
                current_word = line.split(": ", 1)[1]
                lemma_found = False
            elif line.startswith("Лемма:") and current_word:
                parts = line.split(": ", 1)
                if len(parts) > 1:
                    lemma = parts[1].strip()
                    if lemma:
                         lemma_found = True

    if current_word and not lemma_found and len(current_word) >= 4:
         words_without_lemma.append(current_word)

    with open(output_file, 'w', encoding='utf-8') as outfile:
        for word in words_without_lemma:
            outfile.write(word + '\n')


if __name__ == "__main__":
    input_file = 'output_lemmatized_all.txt'
    output_file = 'not_lemma.txt'
    find_words_without_lemma(input_file, output_file)
    print(f"Слова без леммы (не короче 4 символов) записаны в файл '{output_file}'.")

#### **3.1 Нахождение лемм у остатка по нечёткому соответствию**

In [ ]:
!pip install fuzzywuzzy uniparser-urmi python-Levenshtein

очень долгий код далее (примерно 50 мин). Нахождение примерных лемм слов

In [ ]:
import re
from fuzzywuzzy import process
from uniparser_urmi import UrmiAnalyzer
from collections import defaultdict
from tqdm import tqdm

def get_all_urmi_lexemes(lexemes_file):
    """
    Читает лексемы из файла словаря урми и возвращает список лексем.
    """
    lexemes = []
    with open(lexemes_file, 'r', encoding='utf-8') as infile:
      for line in infile:
        if line.strip().startswith("lex: "):
          lexeme = line.strip().replace('lex: ', '')
          lexemes.append(lexeme)
    #print(lexemes)
    return lexemes

def extract_consonants(word):
    """
    Извлекает последовательность согласных из слова.
    """
    return "".join(re.findall(r"[bcdfghjklmnpqrstvwxyz’вƶşçţ]", word, re.IGNORECASE))

def compare_consonant_sequences(word, lexeme):
    """
    Сравнивает последовательности согласных в слове и лексеме с учетом весов.
    Возвращает коэффициент похожести (0-100).
    """
    word_cons = extract_consonants(word) # согласные
    lexeme_cons = extract_consonants(lexeme)
    if not word_cons or not lexeme_cons:
        return 0
    
    total_weight = 0
    weighted_match = 0
    word_idx = 0
    lexeme_idx = 0
    
    while word_idx < len(word_cons) and lexeme_idx < len(lexeme_cons): # пока не закончилось слово
        weight = 1.0 # вес согласной по умолчанию
        if word_idx < 2:
           weight = 3.0 # вес для первой и второй согласной
        elif word_idx < 3:
            weight = 2.0 # вес для третьей согласной
            
        total_weight += weight # общий вес

        if word_cons[word_idx] == lexeme_cons[lexeme_idx]: # совпадают ли согласные (сравниваем по индексам согласные)
          weighted_match += weight  # сюда добавляются взвешенные совпадения
          lexeme_idx += 1

        word_idx+=1
       
    return (weighted_match / total_weight) * 100 if total_weight > 0 else 0

    
def find_best_lemmas(word, urmi_lexemes, limit=10, threshold=60):
    """
    Находит наиболее вероятные леммы для слова, используя нечеткое сравнение
    и сравнение последовательности согласных. Возвращает список лемм.
    """
    # 1. Предварительный отбор с помощью fuzzywuzzy
    preliminary_matches = process.extract(word, urmi_lexemes, limit=limit)
    if not preliminary_matches:
        return []

    best_matches = []
    
    # 2. Сравнение согласных и выбор подходящих лемм
    for lexeme, _ in preliminary_matches:
        score = compare_consonant_sequences(word, lexeme)
        if score >= threshold and len(lexeme) >= len(word) / 3: # и если слово не оч короткое
            best_matches.append((lexeme, score)) # добавляем пару (лемма, скор)
    
    return best_matches #возвращаем список


def find_missing_lemmas(my_words_file, lexemes_file, output_file, limit=10, threshold=60):
    """
    Находит и предлагает леммы для слов, которые не смог распознать UrmiAnalyzer,
    и записывает результаты в файл.
    """
    analyzer = UrmiAnalyzer(mode='nodiacritics')
    urmi_lexemes = get_all_urmi_lexemes(lexemes_file)

    with open(my_words_file, 'r', encoding='utf-8') as infile, \
            open(output_file, 'w', encoding='utf-8') as outfile:
        for word in (line.strip() for line in infile):

            if word in urmi_lexemes:  # Проверка полного совпадения
                outfile.write(f"Слово: {word}, Предложенная лемма: {word} (100.0%)\n")
                continue

            analyses = analyzer.analyze_words(word)
            has_lemma = False
            if analyses:
                for ana in analyses:
                    if ana.lemma:
                        has_lemma = True
                        break
            if not has_lemma:
                best_lemmas = find_best_lemmas(word, urmi_lexemes, limit, threshold)
                if best_lemmas:
                    outfile.write(f"Слово: {word}, Предложенные леммы: {', '.join([f'{lex} ({score:.1f}%)' for lex, score in best_lemmas])}\n")
                else:
                    outfile.write(f"Слово: {word}, Не найдено подходящих лемм\n")


if __name__ == "__main__":
    my_words_file = "not_lemma.txt"
    lexemes_file = "lexemes.txt"
    output_file = "prob_lemmatized_not_lemma.txt"
    find_missing_lemmas(my_words_file, lexemes_file, output_file, limit=10, threshold=60)

#### **3.2 Расфасовка на файл с леммами и проблемными словами.**

In [ ]:
import re
import os

def remove_duplicate_lines(input_file, output_file):
    """
    Читает файл, удаляет дубликаты строк и записывает результат в новый файл.
    """
    unique_lines = set()
    with open(input_file, 'r', encoding='utf-8') as infile:
        for line in infile:
            unique_lines.add(line)

    with open(output_file, 'w', encoding='utf-8') as outfile:
        for line in unique_lines:
            outfile.write(line)

def count_mismatched_chars(word, lemma):
    """
    Считает количество символов, которые есть в лемме, но нет в слове.
    """
    return sum(1 for char in lemma if char not in word)


def choose_best_lemma(word, lemmas_str):
    """
    Выбирает лучшую лемму из списка предложенных.
    """
    lemmas = []
    for item in lemmas_str.split(", "):
      match = re.search(r"(\w+) \((\d+\.?\d*)\%\)", item)
      if match:
        lemmas.append((match.group(1), float(match.group(2))))
  
    if not lemmas:
        return None
    
    # находим максимальный процент схожести
    max_score = max(score for _, score in lemmas)
    
    # Отбираем леммы с максимальным процентом
    best_lemmas = [(lemma, score) for lemma, score in lemmas if score == max_score]

    if len(best_lemmas) == 1:
        return best_lemmas[0]
    
    # Если несколько лемм с максимальным процентом, то выбираем с наименьшим числом несовпадающих букв!
    best_lemma = min(best_lemmas, key=lambda item: count_mismatched_chars(word, item[0]))
    
    return best_lemma
    
def process_lemmatized_file(input_file, all_file, problem_file):
    """
    Читает файл с результатами лемматизации, переносит совпадения в один файл
    и проблемы в другой.
    """
    with open(input_file, 'r', encoding='utf-8') as infile, \
            open(all_file, 'w', encoding='utf-8') as allout, \
            open(problem_file, 'w', encoding='utf-8') as probout:
        
        for line in infile:
            if "Предложенная лемма:" in line:
                allout.write(line)
            elif "Не найдено подходящих лемм" in line:
                probout.write(line)
            elif "Предложенные леммы:" in line:
                match_word = re.search(r"Слово: (\w+),", line)
                if match_word:
                    word = match_word.group(1)
                    match_lemmas = re.search(r"Предложенные леммы: (.+)", line)
                    if match_lemmas:
                        lemmas_str = match_lemmas.group(1)
                        best_lemma_tuple = choose_best_lemma(word, lemmas_str)
                        if best_lemma_tuple:
                           best_lemma, score = best_lemma_tuple
                           if score < 63 and len(word) > 6:
                               continue
                               #probout.write(f"Слово: {word}, Предложенная лемма: {best_lemma} ({score:.1f}%) +\n")
                           else:
                               allout.write(f"Слово: {word}, Предложенная лемма: {best_lemma} ({score:.1f}%)\n")




if __name__ == "__main__":
    input_file = "prob_lemmatized_not_lemma.txt"
    unique_file = "prob_lemmatized_not_lemma_unique.txt"
    all_file = "prob_lemmatized_all.txt"
    problem_file = "problem_lemmatized.txt"
    
    if not os.path.exists(input_file):
      print(f"Ошибка: файл {input_file} не найден.")
      exit(1)

    # без дубликатов записываем в новый файл
    remove_duplicate_lines(input_file, unique_file)

    process_lemmatized_file(unique_file, all_file, problem_file)
    print(f"Результаты записаны в {all_file} и {problem_file}")

#### **4.1 Сохраняем лемматизированные предложения из отдельных разобранных слов.**
   - без стоп-слов (коротких слов)
   - без лишних разборов

In [ ]:
results = []
current_sentence = None
current_lemmas = []

input_file = 'output_lemmatized_all.txt'
output_file = 'output_lemmas_sentences_all_1.txt'
prob_lemmatized_all_file = 'prob_lemmatized_all.txt'


def find_lemma_in_prob_file(token, prob_lemmatized_all_file):
    """
    Ищет лемму для токена в файле prob_lemmatized_all.txt
    """
    with open(prob_lemmatized_all_file, 'r', encoding='utf-8') as prob_file:
        for line in prob_file:
            if line.startswith(f"Слово: {token},"):
               match = re.search(r"Предложенная лемма: ([\w\d]+)", line)
               if match:
                   print(match.group(1))
                   return match.group(1)
    return None
    
with open(input_file, 'r', encoding='utf-8') as infile:
    for line in infile:
        line = line.strip()
        
        if line.startswith("Предложение:"):
            '''
            Если current_sentence не None (т.е предложение уде обработано)
            то текущее предложение и его леммы добавляются в список results, 
            а список current_lemmas очищается для нового предложения.
            '''
            if current_sentence is not None: 
                results.append({"sentence": current_sentence, "lemmas": current_lemmas})
                current_lemmas = []
            current_sentence = line.split(": ", 1)[1]

            
        # иначе обрабатываем предложение:
        elif line.startswith("Слово:"):
            token = None  # для запоминания токена
            lemma_found = False # для запоминания: найдена ли лемма
            
            # извлекаем токен
            token_line = next(infile).strip()  # Читаем следующую строку
            if ": " in token_line:
                token = token_line.split(": ", 1)[1]
            
            # извлекаем лемму
            try:
                lemma_line = next(infile).strip()  # читаем следующую строку
                if ": " in lemma_line:
                    lemma_candidates = lemma_line.split(": ", 1)[1].strip().split('/') # лемма первая от слэша
                    
                    # проверяем наличие леммы
                    if lemma_candidates and len(lemma_candidates[0]) > 0:
                        first_lemma = lemma_candidates[0]
                        if len(first_lemma) >= 4:
                            current_lemmas.append(first_lemma)  # берём первую лемму
                            lemma_found = True
                
            except StopIteration: #пропустить, если след строки нет
                pass
            
            if not lemma_found and token and len(token) >= 4:
                lemma_from_prob = find_lemma_in_prob_file(token, prob_lemmatized_all_file)
                
                if lemma_from_prob:
                   current_lemmas.append(lemma_from_prob)
                else:
                   current_lemmas.append(token)

   
    if current_sentence is not None:
        results.append({"sentence": current_sentence, "lemmas": current_lemmas})
    #print(results)

with open(output_file, 'w', encoding='utf-8') as outfile:
    for item in results:
        #print(item)
        lemma_string = ' '.join(item['lemmas']) 
        outfile.write(lemma_string.strip() + '\n')

**4.2 Перепроверка, чистка итогового файла**

In [ ]:
import re

def filter_lemmas_and_sentences(input_file, output_file):
    """
    Удаляет короткие слова и однословные предложения из файла.
    """
    filtered_lines = []
    with open(input_file, 'r', encoding='utf-8') as infile:
        for line in infile:
            words = line.strip().split()
            filtered_words = [word for word in words if len(word) >= 3]
            if len(filtered_words) > 1:
                filtered_lines.append(" ".join(filtered_words) + "\n")
    
    with open(output_file, 'w', encoding='utf-8') as outfile:
        outfile.writelines(filtered_lines)
        
if __name__ == "__main__":
    input_file = 'output_lemmas_sentences_all_1.txt'
    output_file = 'output_lemmas_sentences_filtered.txt'
    
    filter_lemmas_and_sentences(input_file, output_file)
    print(f"Результаты записаны в {output_file}")

In [ ]:
import re

def count_unique_words(input_file):
    """
    Сколько уникальных слов
    """
    unique_words = set()
    with open(input_file, 'r', encoding='utf-8') as infile:
        for line in infile:
            words = line.strip().split()
            for word in words:
                unique_words.add(word)
    
    return len(unique_words)
if __name__ == "__main__":
    input_file = 'output_lemmas_sentences_filtered.txt'
    unique_word_count = count_unique_words(input_file)
    print(f"Количество уникальных слов в файле {input_file}: {unique_word_count}")

#### **Оцифровка фото**

**Распознавание с помощью tesseract**

In [ ]:
def ocr_image(image_path, lang='tur', output_file=None):
    """Распознает текст на изображении с помощью Tesseract."""
    try:
        image = Image.open(image_path)
        text = pytesseract.image_to_string(image, lang=lang)
        
        if output_file:
            with open(output_file, 'w', encoding='utf-8') as file:
                file.write(text)
            print(f"Текст успешно записан в файл {output_file}")
        
        return text
    except Exception as e:
        print(f"Ошибка при обработке изображения {image_path}: {e}")
        return None

if __name__ == "__main__":
    image_path = '/home/kuklelik/Jupyter/ассирийские файлы/npg_кусочки/фото.png'
    output_file = '/home/kuklelik/Jupyter/recognized_text.txt'
    recognized_text = ocr_image(image_path, 'tur', output_file)
    if recognized_text:
        print(f"Распознанный текст:\n{recognized_text}")


import re

def process_text(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
            
        text = re.sub(r'(?<!\.)\n', ' ', text)
        text = text.replace('-', '')
        text = re.sub(r'[^\w\s\']', '', text)
        text = re.sub(r'([a-z,İ]{1,2})\'', r'\1 ', text.lower())
        text = text.replace("'", '').replace('ä', 'a').replace('ö', 'o').replace('ü', 'u').replace('â', 'a')
        

        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(text)
        
        print(f"Файл {file_path} обработан успешно.")
    except Exception as e:
        print(f"Ошибка при обработке файла {file_path}: {e}")

if __name__ == "__main__":
    file_path = '/home/kuklelik/Jupyter/recognized_text.txt'
    process_text(file_path)


**Создание словаря токенов**

In [ ]:
# словарь для сравнивания
def create_dictionary(input_file, output_file):
    tokens = set()
    
    with open(input_file, 'r', encoding='utf-8') as file:
        for line in file:
            if line.strip().startswith('Токен:'):
                token = line.strip().split(': ')[1]
                tokens.add(token)
    
    with open(output_file, 'w', encoding='utf-8') as dict_file:
        for token in tokens:
            dict_file.write(token + '\n')

input_file = '/home/kuklelik/Jupyter/output_lemmatized_all.txt'
output_file = 'dict_for_tesseract.txt'
create_dictionary(input_file, output_file)


**Корректировочный код**

In [ ]:
import re
from fuzzywuzzy import fuzz, process

def load_dictionary(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read().splitlines()

def load_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def correct_text(detected_text, dictionary):
    sentences = detected_text.split('\n')
    corrected_sentences = []
    
    for sentence in sentences:
        words = sentence.split()
        corrected_words = []
        
        for word in words:
            if len(word) > 3 and any(char in "oabs" for char in word):
                # Проверка сходства слова с проблемными буквами
                similar_length_words = [w for w in dictionary if abs(len(w) - len(word)) <= 1]
                if similar_length_words:
                    best_match_original = process.extractOne(word, similar_length_words)
                    if best_match_original and best_match_original[1] > 90:
                        print(f"Слово '{word}' уже близко к словарю: {best_match_original[0]}")
                        corrected_words.append(best_match_original[0])  # замена на словарное слово
                        continue
                
                #проверяем оба варианта "g" на "q"
                word_with_q = word.replace('g', 'q')
                word_with_ə_ь = word.replace('o', 'ə').replace('a', 'ə').replace('s', 'ə').replace('b', 'ь')
                word_with_ə_ь_q = word_with_q.replace('o', 'ə').replace('a', 'ə').replace('s', 'ə').replace('b', 'ь')
                
                # ищем слова в словаре с той же длиной +-1
                similar_length_words = [w for w in dictionary if abs(len(w) - len(word)) <= 1]
                
                if similar_length_words:
                    #наиболее близкое слово по расстоянию Левенштейна для обоих вариантов
                    best_match_ə_ь = process.extractOne(word_with_ə_ь, similar_length_words)
                    best_match_ə_ь_q = process.extractOne(word_with_ə_ь_q, similar_length_words)
                    
                    if best_match_ə_ь and best_match_ə_ь[1] > 70 and abs(len(word_with_ə_ь) - len(best_match_ə_ь[0])) == 0:
                        corrected_word = best_match_ə_ь[0]
                        print("лучшее ə_ь: ", best_match_ə_ь[0])
                    elif best_match_ə_ь_q and best_match_ə_ь_q[1] > 70 and abs(len(word_with_ə_ь_q) - len(best_match_ə_ь_q[0])) == 0:
                        corrected_word = best_match_ə_ь_q[0]
                        print("лучшее ə_ь_q: ", best_match_ə_ь_q[0])
                    else:
                        corrected_word = word
                else:
                    corrected_word = word
                
                #  оставшиеся "o" заменяем на "ə"
                if corrected_word.endswith('o'):
                    corrected_word = corrected_word[:-1] + 'ə'
                
                corrected_words.append(corrected_word)
            else:
                corrected_words.append(word)
        
        corrected_sentences.append(' '.join(corrected_words))
    
    return '\n'.join(corrected_sentences)


text_path = 'recognized_text.txt'

dictionary_path = 'dict_for_tesseract.txt'

dictionary = load_dictionary(dictionary_path)
detected_text = load_text(text_path)

corrected_text = correct_text(detected_text, dictionary)

print("Исходный текст:\n", detected_text)
print("Исправленный текст:\n", corrected_text)


def save_corrected_text(corrected_text, output_file="recognized_text.txt"):
    """Записывает исправленный текст в файл."""
    try:
        with open(output_file, "w", encoding="utf-8") as f:
            f.write(corrected_text)
        print(f"Исправленный текст записан в файл: {output_file}")
    except Exception as e:
        print(f"Ошибка при записи в файл: {e}")

save_corrected_text(corrected_text)

In [ ]:
# для записи в общий файл разобранного текста
import os

def append_to_log_file(log_file_path, source_file_path):
    try:
        with open(source_file_path, 'r', encoding='utf-8') as source_file:
            source_content = source_file.read()
        
        with open(log_file_path, 'a', encoding='utf-8') as log_file:
            log_file.write(source_content + '\n\n')  # Добавляем пустые строки для разделения записей
        
        print(f"Содержимое {source_file_path} успешно добавлено в {log_file_path}")
    
    except FileNotFoundError:
        print(f"Файл {source_file_path} не найден.")
    
    except Exception as e:
        print(f"Произошла ошибка: {e}")

# Пути к файлам
log_file_path = 'recognized_all.txt'
source_file_path = 'recognized_text.txt'

# Создание файла recognized_all.txt, если он не существует
if not os.path.exists(log_file_path):
    open(log_file_path, 'w').close()
    print(f"Файл {log_file_path} создан.")

# Дозапись содержимого recognized_text.txt в recognized_all.txt
append_to_log_file(log_file_path, source_file_path)

**Сравнительный анализ**

In [ ]:
pip install python-Levenshtein difflib

In [ ]:
import Levenshtein as lev
from difflib import Differ
import re

def preprocess(text):
    # Удаляем пунктуацию, пробелы, приводим к нижнему регистру
    text = re.sub(r'[^\w]', '', text).lower().replace(" ", "")
    return text

def read_file(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        return [preprocess(line) for line in f.readlines()]

text1 = read_file('text1.txt')
text2 = read_file('text2.txt')

# Сравнение по Левенштейну
combined1 = ''.join(text1)
combined2 = ''.join(text2)
distance = lev.distance(combined1, combined2)
similarity = (1 - distance / max(len(combined1), len(combined2))) * 100
print(f"Схожесть после предобработки: {similarity:.1f}%")

# Визуализация различий (на оригинальных строках)
with open('text1.txt', 'r', encoding='utf-8') as f1, open('text2.txt', 'r', encoding='utf-8') as f2:
    orig1 = f1.readlines()
    orig2 = f2.readlines()

d = Differ()
diff = list(d.compare(orig1, orig2))

print("\nОсновные различия в оригинальных текстах:")
for line in diff[:20]:  # Показываем первые 20 различий
    if line.startswith('- ') or line.startswith('+ '):
        print(line.strip())

Схожесть после предобработки: 71.7%

Основные различия в оригинальных текстах:
- hindvəji dərqul d imperialisti
+ Hindvəji dərkul d'imperialisti
- (masi̇s nani ildme)
- narazbjuta min thumə d inolisnsii
+ narazbjuta min huqma d'inglisnji
- go hindustan gul um guş ii
+ go Hindustan qul juma qus gərvusla
- danatb xarajb mtela i̇tixubə xaraja
+ danatb xaraјb mtәla l'tixubo xaraјa,
- ina dureuaza hindustan ka gi
- nəpilvə kodrə tamam gis alma ka
+ ijna burzuazija d'Hindustan kә d'la
+ nәpilva kәdro tamam l'qiѕ alma d'la
- famamta d rbzaja d alma midri qomitet


тяжёлые буквы:

    Ə ə (schwa),
    Ţ ţ (T с седилью снизу),
    Ş ş (S с седилью снизу),
    Ç ç (C с седилью снизу).
    Ь ь

можно пробовать турецкий (НЕТ Ə ə, Ţ ţ, Ь ь), румынский ( НЕТ Ə ə, Ç ç, Ь ь), албанский (НИчего нет, кроме диакритик каких-то похожих)

турецкий:  Ə  распознаёт как "о, а, е, s"
ь как b

Можно попробовать почистить от коротких слов и пр. Токенизировать, лемматизировать. И если леммы не найдётся - попеределывать буквы.  И снова позапускать

### **Для SBOW**

11352 -> 10720 13201 -> 5904

в парсере 3643 у меня 3106 + 2798 = 5904 (побочка 537)


### Обучение

 1/ Сначала он создает **словарь**, «обучаясь» на входных текстовых данных, 
 
 а 2/ затем вычисляет векторное представление слов (**эмбеддинги**).
 
 Векторное представление основывается на *контекстной близости*: слова, *встречающиеся в тексте рядом с одинаковыми словами*
 (а следовательно, согласно дистрибутивной гипотезе, имеющие схожий смысл),
 в векторном представлении будут иметь *близкие координаты векторов-слов*. Для вычисления близости слов используется **косинусное расстояние между их векторами**.

- Сначала прочитайте корпус *построчно* (новая строка = новый документ) и получите список списков, 
где элементами внешнего списка являются документы, а элементами документа являются слова, которые встречаются в этом документе.

In [ ]:
# Возвращает список документов, где каждый документ представлен как список слов.
# Эта функция загружает текстовый корпус из указанного файла и возвращает его в виде списка списков слов. 
# Каждый элемент внешнего списка соответствует строке в файле.

import io

def load_corpus(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    documents = []
    for line in fin:
        documents.append(line.split())
    return documents

- Далее нам нужно сохранить словарь в файл и загрузить его из файла

In [ ]:
# Эта функция сохраняет словарь (где ключи - слова, а значения - их векторные представления) в файл.
# Это необходимо, чтобы сохранить результаты обучения модели.
def save_dictionary(fname, dictionary, args):
    length, dimension = args #длина словаря и размерность векторов
    fin = io.open(fname, 'w', encoding='utf-8')
    fin.write('%d %d\n' % (length, dimension))
    for word in dictionary:
        fin.write('%s %s\n' % (word, ' '.join(map(str, dictionary[word]))))

# Эта функция читает словарь из файла, созданного с помощью функции save_dictionary. 
# Это позволяет загрузить ранее обученную модель.
def load_dictionary(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    length, dimension = map(int, fin.readline().split())
    dictionary = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        dictionary[tokens[0]] = map(float, tokens[1:])
    return dictionary

In [ ]:
documents = load_corpus('output_lemmas_sentences_filtered.txt')
documents

In [ ]:
len(documents)

In [ ]:
model.wv.most_similar('pulxənə', topn = 100)

In [ ]:
save_dictionary('urmi_dictionary.txt', dictionary, (len(dictionary), dimension))

In [ ]:
loaded_dictionary = load_dictionary('urmi_dictionary.txt')
len(dictionary) == len(loaded_dictionary)
# сравнивает длину словаря, только что созданного на основе модели `Word2Vec`, со словарём, загруженным из файла.
# результатом будет `True`, если длина словарей совпадает, и `False` если не совпадает. Это сделано для проверки того,
# что длина обоих словарей совпадает.

    13201 8
    həvi -2.8979242 -2.632021 2.0544846 1.2907332 0.3593198 -0.72183436 4.841792 0.006001376
    вitə -3.1749682 -2.6964538 2.1477616 1.1134115 0.34639588 -0.4612211 4.59261 -0.19204788
    ijli -3.2210557 -2.9326 1.9871407 1.1401004 0.71539384 -0.6838911 5.3129373 -0.15462482
    gənə -3.093706 -2.983472 2.121494 1.4011399 0.4959302 -0.68010485 5.025494 -0.027100356